In [38]:
import pandas as pd
import numpy as np
import re
import pickle
from sklearn.svm import SVC
from sklearn import datasets
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from stop_words import get_stop_words
from joblib import dump, load

# Regex pour garder uniquement les chiffre et les lettres

In [39]:
df1=pd.read_csv("labels.csv")
df1.head(10)

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
5,5,3,1,2,0,1,"!!!!!!!!!!!!!!!!!!""@T_Madison_x: The shit just..."
6,6,3,0,3,0,1,"!!!!!!""@__BrighterDays: I can not just sit up ..."
7,7,3,0,3,0,1,!!!!&#8220;@selfiequeenbri: cause I'm tired of...
8,8,3,0,3,0,1,""" &amp; you might not get ya bitch back &amp; ..."
9,9,3,1,2,0,1,""" @rhythmixx_ :hobbies include: fighting Maria..."


In [40]:
df1.drop(['Unnamed: 0'],axis='columns',inplace=True)

In [41]:
df1.head()

,count,hate_speech,offensive_language,neither,class,tweet
0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [42]:
df1.tweet.head()

0    !!! RT @mayasolovely: As a woman you shouldn't...
1    !!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2    !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3    !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4    !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
Name: tweet, dtype: object

In [43]:
df = pd.read_csv("labels.csv", usecols=['class', 'tweet'])

# Cleannage Data

In [44]:
df['tweet'] = df['tweet'].apply(lambda tweet:re.sub("[^A-Za-z]+", ' ', tweet.lower()))

In [45]:
df.head()

,class,tweet
0,2,rt mayasolovely as a woman you shouldn t comp...
1,1,rt mleew boy dats cold tyga dwn bad for cuffi...
2,1,rt urkindofbrand dawg rt sbaby life you ever ...
3,1,rt c g anderson viva based she look like a tr...
4,1,rt shenikaroberts the shit you hear about me ...


In [46]:
df.tweet.describe()

count                       24783
unique                      24751
top       these hoes ain t loyal 
freq                            4
Name: tweet, dtype: object

# Model Persistence

In [47]:
clf = SVC()

In [48]:
clf = make_pipeline(
    TfidfVectorizer(stop_words=get_stop_words('en')),
    OneVsRestClassifier(SVC(kernel='linear', probability=True))
)

In [49]:
X = df['tweet']
y = df['class']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.80, random_state=42)

In [50]:
clf.fit(X=X_train,y=y_train)

Pipeline(memory=None,
     steps=[('tfidfvectorizer', TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.float64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=u'l2', preprocessor=None, sm...ility=True, random_state=None,
  shrinking=True, tol=0.001, verbose=False),
          n_jobs=None))])

In [51]:
s = pickle.dumps(clf)
clf2 = pickle.loads(s)
print(df['tweet'][0:10])
clf2.predict(X[0:10])

0     rt mayasolovely as a woman you shouldn t comp...
1     rt mleew boy dats cold tyga dwn bad for cuffi...
2     rt urkindofbrand dawg rt sbaby life you ever ...
3     rt c g anderson viva based she look like a tr...
4     rt shenikaroberts the shit you hear about me ...
5     t madison x the shit just blows me claim you ...
6     brighterdays i can not just sit up and hate o...
7     selfiequeenbri cause i m tired of you big bit...
8     amp you might not get ya bitch back amp thats...
9      rhythmixx hobbies include fighting mariam bitch
Name: tweet, dtype: object


array([2, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [52]:
dump(clf,"algo.joblib")

['algo.joblib']